In [3]:
import numpy as np
import pandas as pd
import yfinance as yf

# 1. 數據準備 (這部分比較瑣碎，我先給你)
tickers = ['TSM', 'SPY', 'NVDA']
data = yf.download(tickers, start='2023-01-01', end='2025-01-01', progress=False)['Close']
returns = data.pct_change().dropna()

# 轉換為 Numpy Array
y = returns['TSM'].values
x1 = returns['SPY'].values
x2 = returns['NVDA'].values
N = len(y)

# === 你的 coding 挑戰開始 ===

# 2. 構造設計矩陣 X_normal (TSM ~ Alpha + Beta1*SPY + Beta2*NVDA)
# 需求：X_normal 的 shape 必須是 (N, 3)，第一行全是 1
# 提示：使用 np.ones, np.column_stack
ones = np.ones(N)

# TODO 1: 請寫出一行代碼構造 X_normal
# X_normal = ... 
X_normal = np.column_stack((ones, x1, x2))

print(f"X_normal shape: {X_normal.shape}") # 檢查點：必須是 (N, 3)

# 3. 手刻 OLS 解法
# 公式: theta = (X.T @ X)^-1 @ X.T @ y
# 提示：使用 np.linalg.solve 來解線性方程 Ax = b，其中 A = X.T @ X, b = X.T @ y
# 不要用 inv() 硬算，用 solve() 更準

# TODO 2: 計算 Gram Matrix (X^T * X)
# XtX = ...
XtX = X_normal.T @ X_normal

# TODO 3: 計算 X^T * y
# Xty = ...
Xty = X_normal.T @ y

# TODO 4: 解出 theta
# theta_normal = ...
theta_normal = np.linalg.solve(XtX, Xty)

print(f"Alpha: {theta_normal[0]:.5f}")
print(f"Beta_SPY: {theta_normal[1]:.5f}")
print(f"Beta_NVDA: {theta_normal[2]:.5f}")

# === 破壞測試 ===

# 4. 製造共線性災難
# 需求：製造一個假因子 x_fake，它等於 x1 加上極小的雜訊 (例如 1e-10)
noise = np.random.normal(0, 1e-10, N)

# TODO 5: 創造 x_fake
# x_fake = ...
x_fake = x1 + noise

# TODO 6: 構造崩潰矩陣 X_bad (包含: Intercept, SPY, NVDA, FAKE_SPY)
# X_bad = ...
X_bad = np.column_stack((ones, x1, x2, x_fake))

# TODO 7: 計算 X_bad 的 Condition Number (條件數)
# 提示：使用 np.linalg.cond
# cond_number = ...
cond_number = np.linalg.cond(X_bad)

print(f"Condition Number: {cond_number:.2e}")
XtX_bad = X_bad.T @ X_bad
theta_bad = np.linalg.solve(XtX_bad, X_bad.T @ y)
print("Beta_SPY (Bad):", theta_bad[1])
print("Beta_FAKE (Bad):", theta_bad[3])

X_normal shape: (501, 3)
Alpha: -0.00015
Beta_SPY: 0.74320
Beta_NVDA: 0.34793
Condition Number: 1.42e+10
Beta_SPY (Bad): 632497.4583890615
Beta_FAKE (Bad): -632496.7156710146
